In [1]:
## caged_brach
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

from paths import *
from scpts import manipulation
from scpts import io
from scpts import caged_novo

from basedosdados import Storage, Table, Dataset
import basedosdados as bd

import glob


### Download Dados
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED_AJUSTES/
 - ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/

### Get Urls

In [8]:
download_dict = caged_novo.get_download_links()

Estabelecimentos
Movimentações


In [12]:
download_dict['movimentações']['check_download'].keys()

dict_keys(['202001', '202002'])

### Download Data

In [3]:

import shutil
import urllib.request as request
from contextlib import closing

def download_data(download_link, download_path, filename):
    ## download do arquivo
    with closing(request.urlopen(download_link)) as r:
        with open(os.path.join(download_path, filename), "wb") as f:
            shutil.copyfileobj(r, f)

In [4]:
download_data(
    'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Março/CAGEDMOV202103.7z', 
    '.', 
    'CAGEDMOV202103.7z')